In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
data1 = pd.read_csv('/content/sensor_data_nRF_IMU_1_19092025_164551_not_labelled.csv')
data2 = pd.read_csv('/content/sensor_data_nRF_IMU_2_19092025_164551_labelled.csv')

In [ ]:
data1.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz
0,16:46:12,9.75,0.90,0.88,0.12,0.18,0.00,14.0,-67.0,-44.0
1,16:46:12,9.76,0.92,0.86,0.12,0.18,-0.12,15.0,-67.0,-42.0
2,16:46:12,9.77,0.91,0.87,0.18,0.18,-0.12,14.0,-65.0,-43.0
3,16:46:12,9.77,0.90,0.88,0.12,0.24,-0.12,14.0,-68.0,-45.0
4,16:46:12,9.78,0.91,0.87,0.18,0.06,-0.06,15.0,-68.0,-39.0


In [ ]:
data2.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz,Position,Unnamed: 11
0,16:55:31,-9.20,1.57,1.10,26.61,2.81,-12.08,21,32,-92,NaN,NaN
1,16:55:31,-9.36,1.38,1.25,28.99,-12.08,-14.77,24,33,-88,NaN,NaN
2,16:55:31,-9.37,1.18,1.40,39.92,-15.20,-10.93,26,31,-90,NaN,NaN
3,16:55:31,-9.31,1.37,1.66,39.12,-11.78,-2.44,29,32,-90,NaN,NaN
4,16:55:31,-9.46,1.58,1.74,23.99,-10.01,1.71,29,29,-92,NaN,NaN


In [ ]:
data2 = data2.drop('Unnamed: 11', axis=1)
data2.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz,Position
0,16:55:31,-9.20,1.57,1.10,26.61,2.81,-12.08,21,32,-92,NaN
1,16:55:31,-9.36,1.38,1.25,28.99,-12.08,-14.77,24,33,-88,NaN
2,16:55:31,-9.37,1.18,1.40,39.92,-15.20,-10.93,26,31,-90,NaN
3,16:55:31,-9.31,1.37,1.66,39.12,-11.78,-2.44,29,32,-90,NaN
4,16:55:31,-9.46,1.58,1.74,23.99,-10.01,1.71,29,29,-92,NaN


In [ ]:
data2.dropna(inplace=True)
data2.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz,Position
195,16:55:54,-8.99,1.25,1.24,-19.41,7.63,-5.55,37,28,-92,1.0
196,16:55:54,-10.44,1.07,2.00,-17.70,9.58,15.08,35,31,-88,1.0
197,16:55:54,-10.47,1.48,0.36,-139.34,2.01,-9.64,28,30,-92,1.0
198,16:55:54,-9.37,1.57,-0.37,-3.54,48.46,-33.57,25,30,-89,1.0
199,16:55:54,-12.47,-0.49,0.39,-57.19,-78.55,-24.35,25,29,-93,1.0


In [ ]:
data = pd.merge(data1, data2, on=['TimeStamp'])
data.head()

,TimeStamp,Ax_x,Ay_x,Az_x,Gx_x,Gy_x,Gz_x,Mx_x,My_x,Mz_x,Ax_y,Ay_y,Az_y,Gx_y,Gy_y,Gz_y,Mx_y,My_y,Mz_y,Position
0,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-8.99,1.25,1.24,-19.41,7.63,-5.55,37,28,-92,1.0
1,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-10.44,1.07,2.00,-17.70,9.58,15.08,35,31,-88,1.0
2,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-10.47,1.48,0.36,-139.34,2.01,-9.64,28,30,-92,1.0
3,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-9.37,1.57,-0.37,-3.54,48.46,-33.57,25,30,-89,1.0
4,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-12.47,-0.49,0.39,-57.19,-78.55,-24.35,25,29,-93,1.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15081 entries, 0 to 15080
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TimeStamp  15081 non-null  object 
 1   Ax_x       15081 non-null  float64
 2   Ay_x       15081 non-null  float64
 3   Az_x       15081 non-null  float64
 4   Gx_x       15081 non-null  float64
 5   Gy_x       15081 non-null  float64
 6   Gz_x       15081 non-null  float64
 7   Mx_x       15081 non-null  float64
 8   My_x       15081 non-null  float64
 9   Mz_x       15081 non-null  float64
 10  Ax_y       15081 non-null  float64
 11  Ay_y       15081 non-null  float64
 12  Az_y       15081 non-null  float64
 13  Gx_y       15081 non-null  float64
 14  Gy_y       15081 non-null  float64
 15  Gz_y       15081 non-null  float64
 16  Mx_y       15081 non-null  int64  
 17  My_y       15081 non-null  int64  
 18  Mz_y       15081 non-null  int64  
 19  Position   15081 non-null  float64
dtypes: flo

In [ ]:
X = data.drop(['Position','TimeStamp'], axis=1).values
y = data['Position'].values

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
y

array([ 1.,  1.,  1., ..., 12., 12., 12.])

In [ ]:
y=y-1

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
sequence_length = 50

In [ ]:
def create_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:(i + seq_len)])
        ys.append(y[i + seq_len])
    return np.array(Xs), np.array(ys)

In [ ]:
X_seq, y_seq = create_sequences(X, y, sequence_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 31s 146ms/step - accuracy: 0.4420 - loss: 1.6920 - val_accuracy: 0.7875 - val_loss: 0.5920
Epoch 2/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 28s 150ms/step - accuracy: 0.8034 - loss: 0.5760 - val_accuracy: 0.8723 - val_loss: 0.3907
Epoch 3/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 27s 143ms/step - accuracy: 0.8613 - loss: 0.3980 - val_accuracy: 0.8617 - val_loss: 0.3823
Epoch 4/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 41s 141ms/step - accuracy: 0.8796 - loss: 0.3381 - val_accuracy: 0.9248 - val_loss: 0.2078
Epoch 5/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 26s 140ms/step - accuracy: 0.9137 - loss: 0.2557 - val_accuracy: 0.9238 - val_loss: 0.2233
Epoch 6/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 27s 144ms/step - accuracy: 0.9179 - loss: 0.2427 - val_accuracy: 0.9441 - val_loss: 0.1631
Epoch 7/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 26s 141ms/step - accuracy: 0.9320 - loss: 0.2036 - val_accuracy: 0.9288 - val_loss: 0.2209
Epoch 8/30
188/188 ━━━━━━━━━━━━━━━━━━━━ 27s 141ms/step - accuracy: 0.9306 - loss: 0

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.9809 - loss: 0.0638
Test Accuracy: 0.98


In [ ]:
print("Thank You")

Thank You
